## II. Prepare — What do We Need?

### A. Description of Data

>Available on hands<br>
Content : Details of every ride logged by Cyclistic customers<br>
Range of Data : 2013 - 2024 Mar<br>

> Used in project<br>
Content : Details of every ride logged by Cyclistic customers<br>
Range of Data : 2023 Apr - 2024 Mar `Past 12 months`<br> 


### B. Credibility of Data

The credibility and integrity of our data can be determined using the ROCCC system.

Reliable — it has a large sample size, reflecting the population size.
Original — we can locate the primary source.
Comprehensive — it is understandable and does not contain any missing critical information needed to answer the question or find the solution, nor does it have human error.
Current — it is relevant and up to date, thus indicating that the source refreshes its data regularly.
Cited — the source has been vetted.

### C. Limitations of Data

Data privacy issues prohibit using riders' personally identifiable information such as gender and age, it means that we cannot provide relationship between cutsomers' characteristic such as geographic and demoographic information to customers' behavioural.

Besides, there are no data on hand about pricing. 

## III. Process — From Dirty to Clean

### Decision of tool

Tool: Python and Tableau

In [ ]:
import sys
assert sys.version_info >= (3, 10)

import pandas as pd
import numpy as np

# # To plot figures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt

# import seaborn as sns

# Common imports
import os
print("Libraries imported successfully.")

### Locate the file

In [ ]:
# Define the base directory where the CSV files are stored
directory = r'\input'

# Optionally, define a pattern if the files are consistently named
file_pattern = '-divvy-tripdata.csv'

# Read a file as sample
df_sample = pd.read_csv(r'input\202304-divvy-tripdata.csv')



### Description of the file

In [ ]:
# Generate the basic info. of the sample
df_sample.info()

# In view of the basic info., there are large number of 'null' items in some columns, 
# We can filter some significant column to increase the understanding of the database 
df_sample[df_sample['start_station_name'].notna() & df_sample['end_station_name'].notna()].head(5) 



## Assess the impact of the missing value

In [ ]:
# Calculate the number of rows
def impact_of_missing_value(df):
    total_rows = df.shape[0]

    # Calculate the number of missing values per column
    missing_counts = df.isnull().sum()

    # Calculate the percentage of missing values per column
    missing_percentage = round((missing_counts / total_rows) * 100,2)

    # Calculate the percentage of non-missing values per column
    non_missing_percentage = 100 - missing_percentage

    # Create a DataFrame to nicely display the results
    data_loss_df = pd.DataFrame({
        'Total Rows': total_rows,
        'Missing Values': missing_counts,
        'Percentage Missing': missing_percentage,
        'Percentage Non-Missing': non_missing_percentage
    })

    
    return data_loss_df
print(impact_of_missing_value(df_sample))

In [ ]:
def missing_status_checking(df):
    # Number of cases where 'end_station_name' is missing but 'start_station_name' is not missing
    end_missing_start_not = df[df['end_station_name'].isnull() & df['start_station_name'].notnull()].shape[0]

    # Number of cases where 'start_station_name' is missing but 'end_station_name' is not missing
    start_missing_end_not = df[df['start_station_name'].isnull() & df['end_station_name'].notnull()].shape[0]

    # Number of cases where both 'start_station_name' and 'end_station_name' are missing
    both_missing = df[df['start_station_name'].isnull() & df['end_station_name'].isnull()].shape[0]
    

    print(f"End station name missing, start station name not missing: {end_missing_start_not}")
    print(f"Start station name missing, end station name not missing: {start_missing_end_not}")
    print(f"Both start and end station names missing: {both_missing}")

missing_status_checking(df_sample)

## Check duplicate

In [ ]:
def check_duplicate(df):
    duplicate_status = df.duplicated().any()
    print(f'Duplicate found : {duplicate_status}')
    return

check_duplicate(df_sample)

## Handling null data

Regards `null` value in `end_lat`, we can check on the start point and find any reason for missing.

In [ ]:
count_by_station = df_sample[df_sample['end_lat'].isnull()].groupby('start_station_name').size()

# Sort the counts in descending order
sorted_count_by_station = count_by_station.sort_values(ascending=False)

print(sorted_count_by_station)

In [ ]:
# Replace all NaN values with 'unknown'
def df_fillna_unknown (df):
    df_sample_filled = df.fillna('unknown')
    print(f'Null values items:\n{df_sample_filled.isnull().sum()}')
    return df_sample_filled

df_sample_filled = df_fillna_unknown(df_sample)
print(df_fillna_unknown(df_sample).head())



B. Data Transformation

#### Column on hand

1. `ride_id`: Unique ID assigned with each ride
2. `rideable_type`: Type of bicycle used on each ride — classic, docked, or electric
3. `started_at`: Date and time at the start of each trip
4. `ended_at`: Date and time at the end of each trip
5. `start_station_name`: Name of the station where each journey started from
6. `start_station_id`: ID of the station where each journey started from
7. `end_station_name`: Name of the station where each trip ended at
8. `end_station_id`: ID of the station where each trip ended at
9. `start_lat`: Latitude of each starting station
10. `start_lng`: Longitude of each starting station
11. `end_lat`: Latitude of each ending station
12. `end_lng`: Longitude of each ending station
13. `member_casual`: Type of membership of each rider



#### Additional:
- `ride_length` : Length of each ride
- `ride_length_minutes` : Length of each ride in minutes
- `start_hour` : Time in hour for starting each ride
- `weekday_name` : Weekday of each ride
- `ride_length_minutes_category` : Dividing in different minitues category to understand the usage
- `ride_length_category` : Dividing in diiferent category to understand the purpose of usage

#### Not in used
 - `trip_distance` : There are chance of criculation ride instead of point to point ride. For example, the starting point is so close to the end point such as riding in a park.


In [ ]:

def processing_data(df):
    # Convert 'started_at' and 'ended_at' to datetime
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])

    # Calculate ride length
    df['ride_length'] = df['ended_at'] - df['started_at']

    # Optional: Convert ride length to minutes
    df['ride_length_minutes'] = df['ride_length'].dt.total_seconds() / 60
    
    df['start_hour'] = df['started_at'].dt.hour
    print(df['start_hour'].unique())

    df['weekday_name'] = df['started_at'].dt.day_name()
    print(df['weekday_name'].unique())

    # Filter out rides with duration less than 1 minute or more than 720 minutes
    df_filtered = df[(df['ride_length_minutes'] >= 1) & (df['ride_length_minutes'] <= 720)]

    # Report dropped entries
    dropped_entries = len(df) - len(df_filtered)
    print(f'Entries dropped for being outside 1-720 minutes: {dropped_entries}')
    print(f'Percentage dropped: {100 * dropped_entries / len(df):.2f}%')
    



    # Define bins for the ride length categories
    bins_ride_mins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 90, 120, float('inf')]
    labels_ride_mins = [
        '1-5 mins', '5-10 mins', '10-15 mins', '15-20 mins', 
        '20-25 mins', '25-30 mins', '30-35 mins', '35-40 mins', 
        '40-45 mins', '45-50 mins', '50-55 mins', '55-60 mins', 
        '60-90 mins', '90-120 mins', '120+ mins'
    ]

    # Define the bins and labels
    bins_ride_cate = [0, 5, 15, 30, 60, float('inf')]
    labels_ride_cate = ["Very Short (1-5 mins)", "Short (6-15 mins)", "Moderate (16-30 mins)", "Long (31-60 mins)", "Very Long (60+ mins)"]

    # Create a new column 'ride_length_category'
    df_filtered['ride_length_minutes_category'] = pd.cut(df_filtered['ride_length_minutes'], bins=bins_ride_mins, labels=labels_ride_mins, right=False)
    df_filtered['ride_length_category'] = pd.cut(df_filtered['ride_length_minutes'], bins=bins_ride_cate, labels=labels_ride_cate, right=False)
    return df_filtered


df_sample_filled = processing_data(df_sample_filled)
print(df_sample_filled.head())


## Drop the sample, start to merge the whole dataset

In [ ]:
del df_sample
del df_sample_filled

## `Merge Data`

In [ ]:

# In case you place the whole dataset in difference place
directory = r'\input'

# The same with the sample
file_pattern = '-divvy-tripdata.csv'

dataframes = []  # List to store each DataFrame

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('-divvy-tripdata.csv'):
        print("Processing file:", filename)
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
    else:
        print("Skipping file:", filename)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)
print("Combined DataFrame shape:", combined_df.shape)

# If you want to save the conbined_df as csv
# combined_df.to_csv('combined_data.csv', index=False)

### Reperformance

##### i. Check the dataframe

In [ ]:
combined_df.info()


##### ii. Check the impact of missing value

In [ ]:
print(impact_of_missing_value(combined_df))
missing_status_checking(combined_df)


##### iii. Check duplicate

In [ ]:
check_duplicate(combined_df)


##### iv. Fill the null value with 'unknown'

In [ ]:
print(df_fillna_unknown(combined_df).head())
df_merged_filled = df_fillna_unknown(combined_df)

##### v. Processing the dataframe

In [ ]:
df_merged_filled = processing_data(df_merged_filled)
df_merged_filled.info()
print(df_merged_filled.head())


##### vi. Describe the dataframe

In [ ]:
df_merged_filled.describe(include='all')
print(df_merged_filled.shape)

##### vii. Output the dataframe to csv

In [ ]:
df_merged_filled.to_csv('processed_data.csv', index=False)